### Data saving

In [6]:
import pandas as pd
import numpy as np

In [3]:
# train
df_trans = pd.read_csv('../input/train_transaction.csv')
df_test_trans = pd.read_csv("../input/test_transaction.csv")

# test
df_id = pd.read_csv('../input/train_identity.csv')
df_test_id = pd.read_csv("../input/test_identity.csv")

sample_submission = pd.read_csv("../input/sample_submission.csv",
                               index_col = 'TransactionID')

df_train = df_trans.merge(df_id, how='left', left_index=True,
                         right_index=True, on='TransactionID')
df_test = df_test_trans.merge(df_test_id, how='left', left_index=True,
                         right_index=True, on='TransactionID')

In [4]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(
        end_mem, 100 * (start_mem - end_mem) / start_mem))
    
    return df

In [7]:
%%time
df_train = reduce_mem_usage(df_train)
df_test = reduce_mem_usage(df_test)

Mem. usage decreased to 645.97 Mb (67.0% reduction)
Mem. usage decreased to 561.50 Mb (66.5% reduction)
CPU times: user 4min 29s, sys: 4min 45s, total: 9min 15s
Wall time: 4min 52s


In [8]:
%%time
df_train.to_pickle("../input/df_train.pkl")
df_test.to_pickle("../input/df_test.pkl")

CPU times: user 2.34 s, sys: 2.34 s, total: 4.68 s
Wall time: 5.05 s
